# Gender Recognition using Convolutional Neural Networks 
## ~ Ayush Dey

# Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers as L
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preparation

In [ ]:
df=pd.read_csv("../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

# Distribution and Normalization

In [ ]:
for i in ['age','ethnicity','gender']:
    plt.figure(figsize=(5,5))
    plt.hist(df[i])
    plt.xlabel(i)
    plt.ylabel("count")

In [ ]:
df['pixels']=df['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))

In [ ]:
df['pixels']=df['pixels']/255

# Gender Recognition Model Building

In [ ]:
X=np.array(df['pixels'].tolist())
X=X.reshape(X.shape[0],48,48,1)

In [ ]:
y=df['gender']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
input=X_train.shape[1:]
model = tf.keras.Sequential([
    L.InputLayer(input_shape=input),
    L.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    L.BatchNormalization(),
    L.MaxPooling2D((2, 2)),
    L.Conv2D(64, (3, 3), activation='relu'),
    L.MaxPooling2D((2, 2)),
    L.Flatten(),
    L.Dense(64, activation='relu'),
    L.Dropout(rate=0.5),
    L.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

# Feeding Model & Evaluation

In [ ]:
model.compile(optimizer='sgd',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
callback= tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=8, verbose=0,restore_best_weights=True)
history = model.fit( X_train, y_train, epochs=20, batch_size=64,validation_split=0.1, callbacks=[callback],verbose=1)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test, verbose=2)

# Sample Testing

In [ ]:
predictions=model.predict(X_test)

In [ ]:
gen={0:'Male',1:'Female'}
for i in range(10):
    plt.figure(figsize=(5,5))
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_test[i].reshape(48,48))
    if predictions[i].round(0)==0:
        plt.xlabel("Actual: "+str(gen[y_test.iloc[i]])+'\n'+"Prediction: Male")
    else:
        plt.xlabel("Actual: "+str(gen[y_test.iloc[i]])+'\n'+"Prediction: Female")
plt.show()

# Result
 ### Training Accuracy: 91.8% 
 ### Testing Accuracy: 89.4%